# SPR 2026 - TF-IDF + TabPFN v0.1.9 (Offline)

## ✅ Compatível com competições que exigem Internet OFF

**Este notebook usa TabPFN v0.1.9 com pesos embutidos.**

### Configuração:

1. **NÃO precisa de internet** - funciona offline
2. Adicione o dataset: `carlmcbrideellis/tabpfn-019-whl`
3. Settings → Accelerator → **GPU T4 x2** (recomendado)
4. Run All

### Limitações do TabPFN v1:
- Máximo ~1000 amostras de treino (usamos subsampling)
- Máximo 100 features (usamos SVD para reduzir)
- Apenas classificação (sem regressão)

---

In [ ]:
# =============================================================================
# SPR 2026 - TFIDF + TABPFN v0.1.9 (OFFLINE - SEM INTERNET)
# =============================================================================
# REQUER: Dataset "carlmcbrideellis/tabpfn-019-whl" adicionado ao notebook
# =============================================================================

import subprocess
import sys
import os
import shutil

print("="*60)
print("SPR 2026 - TF-IDF + TabPFN v0.1.9 (OFFLINE)")
print("="*60)

# ==== INSTALAR TABPFN v0.1.9 DO DATASET ====
print("\n[0/5] Instalando TabPFN v0.1.9 do dataset local...")

# Caminho do dataset Kaggle
TABPFN_DATASET = '/kaggle/input/tabpfn-019-whl'
WHEEL_FILE = f'{TABPFN_DATASET}/tabpfn-0.1.9-py3-none-any.whl'
WEIGHTS_FILE = f'{TABPFN_DATASET}/prior_diff_real_checkpoint_n_0_epoch_100.cpkt'

# Verificar se dataset está disponível
if not os.path.exists(WHEEL_FILE):
    raise FileNotFoundError(
        f"Dataset não encontrado: {WHEEL_FILE}\n"
        "Adicione o dataset 'carlmcbrideellis/tabpfn-019-whl' ao seu notebook!"
    )

# Instalar wheel local (sem internet)
subprocess.run([sys.executable, "-m", "pip", "install", WHEEL_FILE, "-q"], check=True)
print(f"    ✓ TabPFN 0.1.9 instalado do wheel local!")

# Copiar pesos para diretório esperado
import site
site_packages = site.getsitepackages()[0]
models_dir = os.path.join(site_packages, 'tabpfn', 'models_diff')
os.makedirs(models_dir, exist_ok=True)
shutil.copy(WEIGHTS_FILE, models_dir)
print(f"    ✓ Pesos copiados para {models_dir}")

# ==== IMPORTS ====
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import warnings
warnings.filterwarnings('ignore')

from tabpfn import TabPFNClassifier
print("    ✓ TabPFN carregado!")

# ==== CONFIGURAÇÕES ====
SEED = 42
SVD_COMPONENTS = 100  # TabPFN v1 tem limite de 100 features
MAX_TRAIN_SIZE = 1000  # TabPFN v1 tem limite de ~1000 amostras
N_ENSEMBLE_CONFIGS = 32  # Número de configurações de ensemble
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
USE_GPU = torch.cuda.is_available()
np.random.seed(SEED)

print(f"    ✓ GPU disponível: {USE_GPU}")

# ==== CARREGAR DADOS ====
print("\n[1/5] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

# ==== TF-IDF ====
print("\n[2/5] Aplicando TF-IDF...")
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train_tfidf = tfidf.fit_transform(train['report'])
X_test_tfidf = tfidf.transform(test['report'])
y_train = train['target'].values
print(f"    TF-IDF esparso: {X_train_tfidf.shape}")

# ==== SVD - CRÍTICO PARA TABPFN (max 100 features) ====
print(f"\n[3/5] Aplicando SVD: {X_train_tfidf.shape[1]} → {SVD_COMPONENTS} features...")
svd = TruncatedSVD(n_components=SVD_COMPONENTS, random_state=SEED)
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)
print(f"    Variância explicada: {svd.explained_variance_ratio_.sum():.2%}")
print(f"    ✅ Shape denso: {X_train_svd.shape}")

# Normalizar (importante para TabPFN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_svd)
X_test = scaler.transform(X_test_svd)

# ==== TABPFN v0.1.9 ====
print(f"\n[4/5] Executando TabPFN v0.1.9 ({'GPU' if USE_GPU else 'CPU'})...")
device = 'cuda' if USE_GPU else 'cpu'

# TabPFN v1 tem limite de ~1000 amostras
# Usamos ensemble com subsampling estratificado
n_ensembles = 5
all_preds = []

splitter = StratifiedShuffleSplit(
    n_splits=n_ensembles, 
    train_size=min(MAX_TRAIN_SIZE, len(X_train)), 
    random_state=SEED
)

for i, (train_idx, _) in enumerate(splitter.split(X_train, y_train)):
    print(f"    Ensemble {i+1}/{n_ensembles}...")
    
    X_subset = X_train[train_idx]
    y_subset = y_train[train_idx]
    
    # TabPFN v0.1.9 API
    model = TabPFNClassifier(
        device=device,
        N_ensemble_configurations=N_ENSEMBLE_CONFIGS
    )
    model.fit(X_subset, y_subset)
    preds = model.predict_proba(X_test)
    all_preds.append(preds)

# Média das probabilidades
avg_probs = np.mean(all_preds, axis=0)
predictions = np.argmax(avg_probs, axis=1)

print("    ✓ TabPFN executado!")

# ==== SUBMISSÃO ====
print("\n[5/5] Gerando submissão...")
submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print("\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())